In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W08`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W09`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W10`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
34439,2021-03-05 14:56:37+00:00,2021-03-05 15:00:15+00:00,Production,0007 of 0023,E2,None,None,KL18,Sun-NI Delivery,Friday Afternoon,iXUsr_Pulaka(8293),2,3.63,2021-10
34440,2021-03-05 15:42:01+00:00,2021-03-05 15:49:06+00:00,Production,0018 of 0023,E2,None,None,KL18,Sun-NI Delivery,Friday Afternoon,iXUsr_Pulaka(8293),2,7.08,2021-10
34441,2021-03-05 10:46:12+00:00,2021-03-05 10:47:01+00:00,Production,0004 of 0005,O4,None,None,KL20,Sun-NI Delivery,Friday Morning,iXUsr_Tutai(7335),2,0.82,2021-10
23183,2021-03-07 07:05:53+00:00,2021-03-07 07:09:59+00:00,Production,0009 of 0009,F2,None,None,KL15,Mon-NI Delivery,Sunday Morning,iXUsr_Franchesca(8424),4,4.10,2021-10
0,2021-03-06 20:39:02+00:00,2021-03-06 20:39:26+00:00,TBD,None,N2,None,None,KL17,None,Saturday Afternoon,DUmmy,0,0.40,2021-10


In [5]:
data_in_use.shape

(66051, 14)